In [1]:
import re

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from graphly.api_client import SparqlClient

In [2]:
ENDPOINT = "https://ld.zazuko.com/query/"

sparql = SparqlClient(ENDPOINT)
sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "property": "<https://ld.stadt-zuerich.ch/statistics/property/>",
    "measure": "<https://ld.stadt-zuerich.ch/statistics/measure/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "ssz": "<https://ld.stadt-zuerich.ch/statistics/>"
})

In [ ]:
# Cafeterias per population
query = """
SELECT ?time ?place ?wikidata_id (SUM(?ast) AS ?count)
WHERE {
  ssz:AST-BTA a cube:Cube;
                  cube:observationSet/cube:observation ?obs.   
  ?obs property:TIME ?time ;     
       property:RAUM ?place_uri ;
       property:BTA <https://ld.stadt-zuerich.ch/statistics/code/BTA5000> ;
                   measure:AST ?ast .
  
  ?place_uri skos:inScheme <https://ld.stadt-zuerich.ch/statistics/scheme/Quartier> ;
             schema:name ?place .

  ?place_uri schema:sameAs ?wikidata_id . 
}
GROUP BY ?time ?place ?wikidata_id
ORDER BY ?place ?time
"""

df = sparql.send_query(query)
df.head()

In [ ]:
# TO DO: get coordinates from wikidata
# Plot cafeterias per district over time



In [4]:
# Restaurants per 10000 inhabitants
query = """
SELECT *
FROM <https://lindas.admin.ch/stadtzuerich/stat>
WHERE {
    {
    SELECT ?time (SUM(?ast) AS ?rest)
    WHERE {
      ssz:AST-BTA a cube:Cube;
                    cube:observationSet/cube:observation ?obs_rest.   
      ?obs_rest property:TIME ?time ;     
           property:RAUM <https://ld.stadt-zuerich.ch/statistics/code/R30000> ;
           property:BTA <https://ld.stadt-zuerich.ch/statistics/code/BTA5000> ;
           measure:AST ?ast . 
    }
     GROUP BY ?time ?place
  }
  {
    SELECT ?time ?pop
    WHERE {
      ssz:BEW a cube:Cube;
                    cube:observationSet/cube:observation ?obs_pop.   
      ?obs_pop property:TIME ?time ;     
           property:RAUM <https://ld.stadt-zuerich.ch/statistics/code/R30000>;
           measure:BEW ?pop
    }
  }  
}
ORDER BY ?time
"""

df = sparql.send_query(query)
df.head()

,time,rest,pop
0,1934-12-31,1328.0,317367.0
1,1935-12-31,1327.0,318981.0
2,1936-12-31,1321.0,319849.0
3,1937-12-31,1321.0,321380.0
4,1938-12-31,1334.0,329780.0


In [16]:
df = df.fillna(method="ffill")
df["rest_quote"] = df["rest"]/df["pop"]*10000

In [17]:
# Number of restaurants per 10 000 inhabitants
px.line(df, x="time", y = "rest_quote")

In [ ]:
# TODO:

#analyse:
#* BES-BTA: Beschäftigte nach Betriebsart
#* BES-BTA-SEX: Beschäftigte nach Betriebsart, Geschlecht